In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1196741,2021-04-19,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1196742,2021-04-20,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1196743,2021-04-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1196744,2021-04-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
32031,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32033,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32035,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32037,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32039,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
737094,2021-04-19,Arkansas,Arkansas,5001,2087,36.00,5000,Arkansas,AR,Arkansas,State,3017804
737096,2021-04-20,Arkansas,Arkansas,5001,2088,36.00,5000,Arkansas,AR,Arkansas,State,3017804
737098,2021-04-21,Arkansas,Arkansas,5001,2088,36.00,5000,Arkansas,AR,Arkansas,State,3017804
737100,2021-04-22,Arkansas,Arkansas,5001,2089,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1196741,2021-04-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1196742,2021-04-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1196743,2021-04-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1196744,2021-04-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-23') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
458,2021-04-23,Snohomish,Washington,35019,563.00,53061,WA,County,822083,4259.79,68.48
914,2021-04-23,Cook,Illinois,527102,10424.00,17031,IL,County,5150233,10234.53,202.40
1369,2021-04-23,Orange,California,269433,4918.00,6059,CA,County,3175692,8484.23,154.86
1823,2021-04-23,Maricopa,Arizona,533368,9813.00,4013,AZ,County,4485414,11891.17,218.78
2277,2021-04-23,Los Angeles,California,1231448,23762.00,6037,CA,County,10039107,12266.51,236.69
...,...,...,...,...,...,...,...,...,...,...,...
1196097,2021-04-23,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08
1196290,2021-04-23,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1196452,2021-04-23,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1196610,2021-04-23,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
458,2021-04-23,Snohomish,Washington,35019,563.00,53061,WA,County,822083,4259.79,68.48,68.48,4259.79
914,2021-04-23,Cook,Illinois,527102,10424.00,17031,IL,County,5150233,10234.53,202.40,202.40,10234.53
1369,2021-04-23,Orange,California,269433,4918.00,6059,CA,County,3175692,8484.23,154.86,154.86,8484.23
1823,2021-04-23,Maricopa,Arizona,533368,9813.00,4013,AZ,County,4485414,11891.17,218.78,218.78,11891.17
2277,2021-04-23,Los Angeles,California,1231448,23762.00,6037,CA,County,10039107,12266.51,236.69,236.69,12266.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196097,2021-04-23,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08,75.08,1951.95
1196290,2021-04-23,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1196452,2021-04-23,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1196610,2021-04-23,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
770725,2021-04-23,Hale,Alabama,2198,75.00,1065,AL,County,14651,15002.39,511.91,511.91,15002.39,1
838895,2021-04-23,Clarke,Alabama,3498,60.00,1025,AL,County,23622,14808.23,254.00,254.00,14808.23,2
603081,2021-04-23,Lowndes,Alabama,1379,53.00,1085,AL,County,9726,14178.49,544.93,544.93,14178.49,3
448746,2021-04-23,Franklin,Alabama,4221,82.00,1059,AL,County,31362,13458.96,261.46,261.46,13458.96,4
534752,2021-04-23,Etowah,Alabama,13729,350.00,1055,AL,County,102268,13424.53,342.24,342.24,13424.53,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134861,2021-04-23,Platte,Wyoming,619,11.00,56031,WY,County,8393,7375.19,131.06,131.06,7375.19,19
735537,2021-04-23,Converse,Wyoming,990,17.00,56009,WY,County,13822,7162.49,122.99,122.99,7162.49,20
914362,2021-04-23,Lincoln,Wyoming,1389,12.00,56023,WY,County,19830,7004.54,60.51,60.51,7004.54,21
1017588,2021-04-23,Niobrara,Wyoming,151,2.00,56027,WY,County,2356,6409.17,84.89,84.89,6409.17,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
603081,2021-04-23,Lowndes,Alabama,1379,53.00,1085,AL,County,9726,14178.49,544.93,544.93,14178.49,3,1
770725,2021-04-23,Hale,Alabama,2198,75.00,1065,AL,County,14651,15002.39,511.91,511.91,15002.39,1,2
256547,2021-04-23,Walker,Alabama,7167,278.00,1127,AL,County,63521,11282.88,437.65,437.65,11282.88,28,3
533172,2021-04-23,Clay,Alabama,1522,56.00,1027,AL,County,13235,11499.81,423.12,423.12,11499.81,23,4
650358,2021-04-23,Greene,Alabama,916,34.00,1063,AL,County,8111,11293.31,419.18,419.18,11293.31,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648785,2021-04-23,Hot Springs,Wyoming,366,3.00,56017,WY,County,4413,8293.68,67.98,67.98,8293.68,17,19
914362,2021-04-23,Lincoln,Wyoming,1389,12.00,56023,WY,County,19830,7004.54,60.51,60.51,7004.54,21,20
891588,2021-04-23,Uinta,Wyoming,2158,12.00,56041,WY,County,20226,10669.44,59.33,59.33,10669.44,5,21
255344,2021-04-23,Teton,Wyoming,3719,9.00,56039,WY,County,23464,15849.81,38.36,38.36,15849.81,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4259.79,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4259.79,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4259.79,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4259.79,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4259.79,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193934,2021-04-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1193935,2021-04-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1193936,2021-04-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1193937,2021-04-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
767529,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15002.39,2,1,1.00
767530,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15002.39,2,1,0.00
767531,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15002.39,2,1,0.00
767532,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15002.39,2,1,0.00
767533,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,15002.39,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929646,2021-04-19,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
929647,2021-04-20,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
929648,2021-04-21,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
929649,2021-04-22,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
600665,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14178.49,1,3,1.00,0.00
600666,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14178.49,1,3,0.00,0.00
600667,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14178.49,1,3,0.00,0.00
600668,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14178.49,1,3,0.00,0.00
600669,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14178.49,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598690,2021-04-19,Albany,Wyoming,4137,11.00,56001,WY,County,38880,10640.43,28.29,28.29,10761.32,23,4,18.00,0.00
598691,2021-04-20,Albany,Wyoming,4148,11.00,56001,WY,County,38880,10668.72,28.29,28.29,10761.32,23,4,11.00,0.00
598692,2021-04-21,Albany,Wyoming,4155,11.00,56001,WY,County,38880,10686.73,28.29,28.29,10761.32,23,4,7.00,0.00
598693,2021-04-22,Albany,Wyoming,4170,11.00,56001,WY,County,38880,10725.31,28.29,28.29,10761.32,23,4,15.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-23') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
306239,2021-04-23,Imperial,California,28096,719.00,6025,CA,County,181215,15504.24,396.77,396.77,15504.24,1,2,44.00,1.00
2277,2021-04-23,Los Angeles,California,1231448,23762.00,6037,CA,County,10039107,12266.51,236.69,236.69,12266.51,2,5,424.00,26.00
152515,2021-04-23,San Bernardino,California,295390,4571.00,6071,CA,County,2180085,13549.47,209.67,209.67,13549.47,3,4,75.00,-1.00
536284,2021-04-23,Inyo,California,1394,36.00,6027,CA,County,18039,7727.70,199.57,199.57,7727.70,4,25,2.00,0.00
68213,2021-04-23,Stanislaus,California,60753,1036.00,6099,CA,County,550660,11032.76,188.14,188.14,11032.76,5,9,76.00,1.00
31617,2021-04-23,Riverside,California,297881,4559.00,6065,CA,County,2470546,12057.29,184.53,184.53,12057.29,6,6,111.00,4.00
84570,2021-04-23,Tulare,California,49493,832.00,6107,CA,County,466195,10616.37,178.47,178.47,10616.37,7,10,24.00,1.00
54275,2021-04-23,San Joaquin,California,71972,1340.00,6077,CA,County,762148,9443.31,175.82,175.82,9443.31,8,19,73.00,0.00
30791,2021-04-23,Fresno,California,100864,1657.00,6019,CA,County,999101,10095.48,165.85,165.85,10095.48,9,14,26.00,3.00
486009,2021-04-23,Merced,California,31400,454.00,6047,CA,County,277680,11307.98,163.50,163.50,11307.98,10,8,42.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1134412,2021-04-23,Lassen,California,5685,1.00,6035,CA,County,30573,18594.84,3.27,3.27,18594.84,56,1,6.00,0.00
306239,2021-04-23,Imperial,California,28096,719.00,6025,CA,County,181215,15504.24,396.77,396.77,15504.24,1,2,44.00,1.00
650300,2021-04-23,Kings,California,22821,246.00,6031,CA,County,152940,14921.54,160.85,160.85,14921.54,11,3,10.00,0.00
152515,2021-04-23,San Bernardino,California,295390,4571.00,6071,CA,County,2180085,13549.47,209.67,209.67,13549.47,3,4,75.00,-1.00
2277,2021-04-23,Los Angeles,California,1231448,23762.00,6037,CA,County,10039107,12266.51,236.69,236.69,12266.51,2,5,424.00,26.00
31617,2021-04-23,Riverside,California,297881,4559.00,6065,CA,County,2470546,12057.29,184.53,184.53,12057.29,6,6,111.00,4.00
191745,2021-04-23,Kern,California,108195,1345.00,6029,CA,County,900202,12018.97,149.41,149.41,12018.97,14,7,64.00,10.00
486009,2021-04-23,Merced,California,31400,454.00,6047,CA,County,277680,11307.98,163.50,163.50,11307.98,10,8,42.00,0.00
68213,2021-04-23,Stanislaus,California,60753,1036.00,6099,CA,County,550660,11032.76,188.14,188.14,11032.76,5,9,76.00,1.00
84570,2021-04-23,Tulare,California,49493,832.00,6107,CA,County,466195,10616.37,178.47,178.47,10616.37,7,10,24.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
306239,2021-04-23,Imperial,California,28096,719.00,6025,CA,County,181215,15504.24,396.77,396.77,15504.24,1,2,44.00,1.00
2277,2021-04-23,Los Angeles,California,1231448,23762.00,6037,CA,County,10039107,12266.51,236.69,236.69,12266.51,2,5,424.00,26.00
152515,2021-04-23,San Bernardino,California,295390,4571.00,6071,CA,County,2180085,13549.47,209.67,209.67,13549.47,3,4,75.00,-1.00
536284,2021-04-23,Inyo,California,1394,36.00,6027,CA,County,18039,7727.70,199.57,199.57,7727.70,4,25,2.00,0.00
68213,2021-04-23,Stanislaus,California,60753,1036.00,6099,CA,County,550660,11032.76,188.14,188.14,11032.76,5,9,76.00,1.00
31617,2021-04-23,Riverside,California,297881,4559.00,6065,CA,County,2470546,12057.29,184.53,184.53,12057.29,6,6,111.00,4.00
84570,2021-04-23,Tulare,California,49493,832.00,6107,CA,County,466195,10616.37,178.47,178.47,10616.37,7,10,24.00,1.00
54275,2021-04-23,San Joaquin,California,71972,1340.00,6077,CA,County,762148,9443.31,175.82,175.82,9443.31,8,19,73.00,0.00
30791,2021-04-23,Fresno,California,100864,1657.00,6019,CA,County,999101,10095.48,165.85,165.85,10095.48,9,14,26.00,3.00
486009,2021-04-23,Merced,California,31400,454.00,6047,CA,County,277680,11307.98,163.50,163.50,11307.98,10,8,42.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.77,15504.24,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.77,15504.24,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.77,15504.24,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.77,15504.24,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.77,15504.24,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4012,04/19/21,Lassen,5669,1.00,18542.50,3.27,3.27,18594.84,56,1,0.00,0.00
4013,04/20/21,Lassen,5673,1.00,18555.59,3.27,3.27,18594.84,56,1,4.00,0.00
4014,04/21/21,Lassen,5679,1.00,18575.21,3.27,3.27,18594.84,56,1,6.00,0.00
4015,04/22/21,Lassen,5679,1.00,18575.21,3.27,3.27,18594.84,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.77,15504.24,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.77,15504.24,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.77,15504.24,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.77,15504.24,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.77,15504.24,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4098,04/19/21,Merced,31267,452.00,11260.08,162.78,163.50,11307.98,10,8,71.00,0.00
4099,04/20/21,Merced,31289,453.00,11268.01,163.14,163.50,11307.98,10,8,22.00,1.00
4100,04/21/21,Merced,31321,454.00,11279.53,163.50,163.50,11307.98,10,8,32.00,1.00
4101,04/22/21,Merced,31358,454.00,11292.86,163.50,163.50,11307.98,10,8,37.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)